# NumPy Universal Functions

The key to making NumPy computation fast is to use __vectorized* operations__, generally implemented through NumPy's __universal functions__ (ufuncs).

### The Slowness of Loops

Python's default implementation (known as CPython) does some operations very slowly. This is in part due to the interpreted nature of the language - some operations cannot be compiled down to machine code as in languages like C and Fortran.

Recently there have been attempts to address this issue
    * [PyPy](http://pypy.org/) a just-in-time compiled implementation of Python
    * [Cython](http://cython.org) converts Python code to compilable C code
    * [Numba](http://numba.pydata.org/) converts snippets of Python code to fast LLVM bytecode.
    
Each has its strengths and weaknesses, but it is safe to say that none of the three approaches has yet surpassed the reach and popularity of the standard CPython engine.

Imagine we have an array of values and we'd like to compute the reciprocal of each.

In [1]:
import numpy as np
np.random.seed(0)

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output
        
values = np.random.randint(1, 10, size=5)
compute_reciprocals(values)

array([ 0.16666667,  1.        ,  0.25      ,  0.25      ,  0.125     ])

If we benchmark this code for a large input, we see that this operation is VERY slow.

In [2]:
big_array = np.random.randint(1, 100, size=1000000)
%timeit compute_reciprocals(big_array)

1.85 s ± 32.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


The bottleneck isn't the operations, but the type-checking and function dispatches that CPython must do at each cycle of the loop.

Each time the reciprocal is computed, __Python first examines the object's type and does a dynamic lookup of the correct function to use for that type.__

If we were working in compiled code instead, this type specification would be known before the code executes and the result could be computed much more efficiently.

### Introducing UFuncs

UFunc's vectorized approach is designed to push any loops into the compiled layer that underlies NumPy, leading to much faster execution. Compare the results of the following two:

In [3]:
print(compute_reciprocals(values))
print(1.0 / values)

[ 0.16666667  1.          0.25        0.25        0.125     ]
[ 0.16666667  1.          0.25        0.25        0.125     ]


Compare this benchmark to above:

In [4]:
%timeit (1.0 / big_array)

2.74 ms ± 14 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


__Vectorized operations in NumPy are implemented via *ufuncs*__, whose main purpose is to quickly execute repeated operations on values in NumPy arrays.

Ufuncs can also operate between two arrays:

In [5]:
np.arange(5) / np.arange(1, 6)

array([ 0.        ,  0.5       ,  0.66666667,  0.75      ,  0.8       ])

UFunc operations can act on multi-dimensional arrays:

In [6]:
x = np.arange(9).reshape((3, 3))
2 ** x

array([[  1,   2,   4],
       [  8,  16,  32],
       [ 64, 128, 256]])

### Exploring NumPy's UFuncs

Ufuncs exist in two flavors: __unary ufuncs__, which operate on a single input, and __binary ufuncs__, which operate on two inputs.

### Array arithmetic

NumPy's ufuncs feel very natural to use because they make use of Python's native arithmetic operators.
The standard addition, subtraction, multiplication, and division can all be used:

In [7]:
x = np.arange(4)
print("x     =", x)
print("x + 5 =", x + 5)
print("x - 5 =", x - 5)
print("x * 2 =", x * 2)
print("x / 2 =", x / 2)
print("x // 2 =", x // 2)  # floor division

x     = [0 1 2 3]
x + 5 = [5 6 7 8]
x - 5 = [-5 -4 -3 -2]
x * 2 = [0 2 4 6]
x / 2 = [ 0.   0.5  1.   1.5]
x // 2 = [0 0 1 1]


There is also a unary ufunc for negation, and a ``**`` operator for exponentiation, and a ``%`` operator for modulus:

In [8]:
print("-x     = ", -x)
print("x ** 2 = ", x ** 2)
print("x % 2  = ", x % 2)

-x     =  [ 0 -1 -2 -3]
x ** 2 =  [0 1 4 9]
x % 2  =  [0 1 0 1]


In addition, these can be strung together however you wish, and the standard order of operations is respected:

In [9]:
-(0.5*x + 1) ** 2

array([-1.  , -2.25, -4.  , -6.25])

Each of these arithmetic operations are wrappers around NumPy functions. For example, the ``+`` operator is a wrapper for the ``add`` function:

In [10]:
np.add(x, 2)

array([2, 3, 4, 5])

The following table lists the arithmetic operators implemented in NumPy:

| Operator	    | Equivalent ufunc    | Description                           |
|---------------|---------------------|---------------------------------------|
|``+``          |``np.add``           |Addition (e.g., ``1 + 1 = 2``)         |
|``-``          |``np.subtract``      |Subtraction (e.g., ``3 - 2 = 1``)      |
|``-``          |``np.negative``      |Unary negation (e.g., ``-2``)          |
|``*``          |``np.multiply``      |Multiplication (e.g., ``2 * 3 = 6``)   |
|``/``          |``np.divide``        |Division (e.g., ``3 / 2 = 1.5``)       |
|``//``         |``np.floor_divide``  |Floor division (e.g., ``3 // 2 = 1``)  |
|``**``         |``np.power``         |Exponentiation (e.g., ``2 ** 3 = 8``)  |
|``%``          |``np.mod``           |Modulus/remainder (e.g., ``9 % 4 = 1``)|


### Absolute value

NumPy understands Python's built-in absolute value function:

In [11]:
x = np.array([-2, -1, 0, 1, 2])
abs(x)

array([2, 1, 0, 1, 2])

The corresponding NumPy ufunc is ``np.absolute``, which is also available under the alias ``np.abs``:

In [12]:
np.absolute(x)

array([2, 1, 0, 1, 2])

In [13]:
np.abs(x)

array([2, 1, 0, 1, 2])

``abs`` can handle complex data - the absolute value returns the magnitude:

In [14]:
x = np.array([3 - 4j, 4 - 3j, 2 + 0j, 0 + 1j])
np.abs(x)

array([ 5.,  5.,  2.,  1.])

### Trigonometric functions

Start by defining an array of angles:

In [16]:
theta = np.linspace(0, np.pi, 3)
theta

array([ 0.        ,  1.57079633,  3.14159265])

In [17]:
print("theta      = ", theta)
print("sin(theta) = ", np.sin(theta))
print("cos(theta) = ", np.cos(theta))
print("tan(theta) = ", np.tan(theta))

theta      =  [ 0.          1.57079633  3.14159265]
sin(theta) =  [  0.00000000e+00   1.00000000e+00   1.22464680e-16]
cos(theta) =  [  1.00000000e+00   6.12323400e-17  -1.00000000e+00]
tan(theta) =  [  0.00000000e+00   1.63312394e+16  -1.22464680e-16]


The values are computed using machine precision - values that should be zero do not always hit exactly zero.
Inverse trigonometric functions are also available:

In [18]:
x = [-1, 0, 1]
print("x         = ", x)
print("arcsin(x) = ", np.arcsin(x))
print("arccos(x) = ", np.arccos(x))
print("arctan(x) = ", np.arctan(x))

x         =  [-1, 0, 1]
arcsin(x) =  [-1.57079633  0.          1.57079633]
arccos(x) =  [ 3.14159265  1.57079633  0.        ]
arctan(x) =  [-0.78539816  0.          0.78539816]


### Exponents and logarithms

In [19]:
x = [1, 2, 3]
print("x     =", x)
print("e^x   =", np.exp(x))
print("2^x   =", np.exp2(x))
print("3^x   =", np.power(3, x))

x     = [1, 2, 3]
e^x   = [  2.71828183   7.3890561   20.08553692]
2^x   = [ 2.  4.  8.]
3^x   = [ 3  9 27]


The inverse of the exponentials, the logarithms, are also available.
The basic ``np.log`` gives the natural logarithm; if you prefer to compute the base-2 logarithm or the base-10 logarithm, these are available as well:

In [20]:
x = [1, 2, 4, 10]
print("x        =", x)
print("ln(x)    =", np.log(x))
print("log2(x)  =", np.log2(x))
print("log10(x) =", np.log10(x))

x        = [1, 2, 4, 10]
ln(x)    = [ 0.          0.69314718  1.38629436  2.30258509]
log2(x)  = [ 0.          1.          2.          3.32192809]
log10(x) = [ 0.          0.30103     0.60205999  1.        ]


There are also some specialized versions that are __useful for maintaining precision with very small inputs__:

In [22]:
x = [0, 0.001, 0.01, 0.1]
print("exp(x) - 1 =", np.expm1(x))
print("log(1 + x) =", np.log1p(x))

exp(x) - 1 = [ 0.          0.0010005   0.01005017  0.10517092]
log(1 + x) = [ 0.          0.0009995   0.00995033  0.09531018]


### Specialized ufuncs

NumPy has many more ufuncs available, including __hyperbolic trig__ functions, __bitwise arithmetic__, __comparison operators__, __conversions from radians to degrees__, __rounding and remainders__, and much more.

__Another excellent source for more specialized and obscure ufuncs is the submodule ``scipy.special``.__

In [23]:
from scipy import special

In [24]:
# Gamma functions (generalized factorials) and related functions
x = [1, 5, 10]
print("gamma(x)     =", special.gamma(x))
print("ln|gamma(x)| =", special.gammaln(x))
print("beta(x, 2)   =", special.beta(x, 2))

gamma(x)     = [  1.00000000e+00   2.40000000e+01   3.62880000e+05]
ln|gamma(x)| = [  0.           3.17805383  12.80182748]
beta(x, 2)   = [ 0.5         0.03333333  0.00909091]


In [25]:
# Error function (integral of Gaussian)
# its complement, and its inverse
x = np.array([0, 0.3, 0.7, 1.0])
print("erf(x)  =", special.erf(x))
print("erfc(x) =", special.erfc(x))
print("erfinv(x) =", special.erfinv(x))

erf(x)  = [ 0.          0.32862676  0.67780119  0.84270079]
erfc(x) = [ 1.          0.67137324  0.32219881  0.15729921]
erfinv(x) = [ 0.          0.27246271  0.73286908         inf]


### Specifying output

For large calculations, it is sometimes useful to specify the array where the result of the calculation will be stored.
Rather than creating a temporary array, __this can be used to write computation results directly to the memory location where you'd like them to be.__
For all ufuncs, this can be done using the ``out`` argument of the function:

In [26]:
x = np.arange(5)
y = np.empty(5)
np.multiply(x, 10, out=y)
y

array([  0.,  10.,  20.,  30.,  40.])

This can be used with array views. For example, we can write the results of a computation to every other element of a specified array:

In [27]:
y = np.zeros(10)
np.power(2, x, out=y[::2])
y

array([  1.,   0.,   2.,   0.,   4.,   0.,   8.,   0.,  16.,   0.])

If you use ``y[::2] = 2**x``, this would create a temporary array to hold the results of ``2**x``, followed by a second operation to copy those values into the ``y``. 
For very large arrays the memory savings from careful use of the ``out`` argument can be significant.

### Aggregates

For example: __to reduce an array with a particular operation__ we can use the ``reduce`` method of any ufunc.
A reduce repeatedly applies a given operation to the elements of an array until only a single result remains.

For example, calling ``reduce`` on the ``add`` ufunc returns the sum of all elements in the array:

In [28]:
x = np.arange(1, 6)
np.add.reduce(x)

15

Similarly, calling ``reduce`` on the ``multiply`` ufunc results in the product of all array elements:

In [29]:
np.multiply.reduce(x)

120

Use ``accumulate`` to store all the intermediate results of the computation:

In [30]:
np.add.accumulate(x)

array([ 1,  3,  6, 10, 15])

In [31]:
np.multiply.accumulate(x)

array([  1,   2,   6,  24, 120])

### Outer products

Any ufunc can __compute the output of all pairs of two different inputs__ using the ``outer`` method. This allows you, in one line, to do things like create a multiplication table:

In [32]:
x = np.arange(1, 6)
np.multiply.outer(x, x)

array([[ 1,  2,  3,  4,  5],
       [ 2,  4,  6,  8, 10],
       [ 3,  6,  9, 12, 15],
       [ 4,  8, 12, 16, 20],
       [ 5, 10, 15, 20, 25]])